In [31]:
import pandas
import io
from urllib.request import urlretrieve
import spacy
from pypdf import PdfReader
import re

In [5]:
data = pandas.read_xml("2024FD.xml")
data.head()

,Prefix,Last,First,Suffix,FilingType,StateDst,Year,FilingDate,DocID
0,None,Abel,William,None,C,TX31,2024,5/12/2024,10060658
1,None,Abolfazli,Maryam.,None,C,TN05,2024,5/13/2024,10060479
2,Mr.,Aboujaoude,Rock Adel,Jr.,C,FL12,2024,5/14/2024,10061019
3,None,Adams,Jennifer,None,C,FL07,2024,5/30/2024,10062032
4,None,Adams,Sheryl L.,None,W,CA49,2024,4/22/2024,7940


In [10]:
# Date is MM/DD/YYYY
sorted_by_date = data.sort_values(by="FilingDate", ascending=False)
sorted_by_date.head()

,Prefix,Last,First,Suffix,FilingType,StateDst,Year,FilingDate,DocID
523,Hon.,Greene,Marjorie Taylor Mrs,None,P,GA14,2024,9/9/2024,20025821
878,Hon.,Matsui,Doris O.,None,P,CA07,2024,9/9/2024,20025820
300,Hon.,DelBene,Suzan K.,None,P,WA01,2024,9/9/2024,20025768
334,Hon.,Doggett,Lloyd,None,P,TX37,2024,9/9/2024,20025822
1080,None,Puffett,Jody Marie,None,D,IA02,2024,9/7/2024,8220633


In [9]:
sorted_by_DocId = data.sort_values(by='DocID', ascending=False)
sorted_by_DocId.head()

,Prefix,Last,First,Suffix,FilingType,StateDst,Year,FilingDate,DocID
934,None,Meyer,Joe,None,D,OR03,2024,10/18/2024,40003692
1184,None,Shea,Cornelius,None,D,MA02,2024,8/22/2024,40003686
1341,None,Woodward,Nathaniel E.,None,D,UT02,2024,8/13/2024,40003683
1231,None,Summerlin,Sadi,None,D,LA03,2024,8/13/2024,40003682
1201,None,Smiley,Heather,None,D,MI06,2024,8/6/2024,40003681


In [18]:
# DocID must stat with 200
filtered = sorted_by_DocId[sorted_by_DocId['DocID'].astype(str).str.startswith('200')]
filtered.head()

,Prefix,Last,First,Suffix,FilingType,StateDst,Year,FilingDate,DocID
530,Hon.,Greene,Marjorie Taylor Mrs,None,P,GA14,2024,11/21/2024,20026257
1303,Hon.,Wasserman Schultz,Debbie,None,P,FL25,2024,11/20/2024,20026247
48,Hon.,Auchincloss,Jake,None,P,MA04,2024,11/21/2024,20026246
303,Hon.,DelBene,Suzan K.,None,P,WA01,2024,11/21/2024,20026238
949,Hon.,Miller,Max,None,P,OH07,2024,11/15/2024,20026231


In [21]:
# Download the latest
latest = filtered.iloc[0]
url = f"https://disclosures-clerk.house.gov/public_disc/ptr-pdfs/2024/{latest["DocID"]}.pdf"
urlretrieve(url, "latest.pdf")

('latest.pdf', <http.client.HTTPMessage at 0x1f82a57baa0>)

In [ ]:
# NLP shit
nlp = spacy.load("en_core_web_trf")
reader = PdfReader("latest.pdf")

# Pull all the text
text = ""
for x in range(len(reader.pages)):
    page = reader.pages[x]
    text += page.extract_text()

print(len(text))

c:\Users\spaze\AppData\Local\Programs\Python\Python312\Lib\site-packages\thinc\shims\pytorch.py:261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torc

1831


In [34]:
# Parse transactions form thte text

# doc = nlp(text)
# lines = text.splitlines()
# parsed_data = []
# for i, line in enumerate(lines):
#     if "P" in line:
#         try:
#             asset_line = lines[i - 1].strip()
#             details = line.split()
#             asset = asset_line
#             transaction_type = details[0]
#             date = details[1]
#             notification_date = details[2]
#             amount = " ".join(details[3])
#             parsed_data.append({
#                 "Asset": asset,
#                 "Transaction_Type": transaction_type,
#                 "Date": date,
#                 "Notification": notification_date,
#                 "Amount": amount,
#             })
#         except IndexError:
#             continue

# data = pandas.DataFrame(parsed_data)
# print(parsed_data)

doc = nlp(text)
lines = text.splitlines()

asset_pattern = r"^(.*?)-.*Stock \((.*?)\).*"
transaction_pattern = r"^(P|S)"
date_pattern = r"(\d{2}/\d{2}/\d{4})"
amount_pattern = r"(\$\d{1,3}(,\d{3})*(\.\d{2})?( - \$\d{1,3}(,\d{3})*(\.\d{2})?)?)"

transactions = []
for i, line in enumerate(lines):
    if not re.search(date_pattern, line):
        continue
    
    previous_line = lines[i - 1].strip() if i > 0 else ""
    asset_match = re.search(asset_pattern, previous_line)
    asset = asset_match.group(1).strip() if asset_match else previous_line
    
    transaction_match = re.search(transaction_pattern, line)
    date_match = re.findall(date_pattern, line)
    amount_match = re.search(amount_pattern, line)
    
    transaction_type = transaction_match.group(0) if transaction_match else ""
    date = date_match[0] if date_match else ""
    notification_date = date_match[1] if len(date_match) > 1 else ""
    amount = amount_match.group(0) if amount_match else ""
    
    transactions.append({
        "Asset": asset,
        "Transaction Type": transaction_type,
        "Date": date,
        "Notification Date": notification_date,
        "Amount": amount,
    })

print(transactions)
txs = pandas.DataFrame(transactions)
txs.head()

[{'Asset': '[ST]', 'Transaction Type': 'P', 'Date': '11/20/2024', 'Notification Date': '11/21/2024', 'Amount': '$1,001 - $15,000'}, {'Asset': '[ST]', 'Transaction Type': 'P', 'Date': '11/20/2024', 'Notification Date': '11/21/2024', 'Amount': '$1,001 - $15,000'}, {'Asset': '(AMZN) [ST]', 'Transaction Type': 'P', 'Date': '11/20/2024', 'Notification Date': '11/21/2024', 'Amount': '$1,001 - $15,000'}, {'Asset': 'Stock (AMAT) [ST]', 'Transaction Type': 'P', 'Date': '11/20/2024', 'Notification Date': '11/21/2024', 'Amount': '$1,001 - $15,000'}, {'Asset': 'F\x00\x00\x00\x00\x00 S\x00\x00\x00\x00\x00: New', 'Transaction Type': '', 'Date': '11/20/2024', 'Notification Date': '11/21/2024', 'Amount': '$1,001 - $15,000'}, {'Asset': 'Common Stock (OXY) [ST]', 'Transaction Type': 'P', 'Date': '11/20/2024', 'Notification Date': '11/21/2024', 'Amount': '$1,001 - $15,000'}, {'Asset': 'Common Stock (QCOM) [ST]', 'Transaction Type': 'P', 'Date': '11/20/2024', 'Notification Date': '11/21/2024', 'Amount': '

,Asset,Transaction Type,Date,Notification Date,Amount
0,[ST],P,11/20/2024,11/21/2024,"$1,001 - $15,000"
1,[ST],P,11/20/2024,11/21/2024,"$1,001 - $15,000"
2,(AMZN) [ST],P,11/20/2024,11/21/2024,"$1,001 - $15,000"
3,Stock (AMAT) [ST],P,11/20/2024,11/21/2024,"$1,001 - $15,000"
4,F      S     : New,,11/20/2024,11/21/2024,"$1,001 - $15,000"
